In [12]:
import pandas as pd
import re
from string import punctuation as pun
from chat_word_dict import abbreviations
from textblob import TextBlob
from gensim.parsing.preprocessing import STOPWORDS

In [13]:
data = pd.read_csv('data.csv', encoding='utf-8', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text'])

In [14]:
data.drop(columns=['ids', 'date', 'flag', 'user'], inplace=True)

In [15]:
data

,target,text
0,id,NaN
1,1,NaN
2,4,NaN
3,5,NaN
4,6,NaN
...,...,...
7609,10869,NaN
7610,10870,NaN
7611,10871,NaN
7612,10872,NaN


PreProcessing:

Lower-Casing:

In [16]:
data['text'] = data['text'].str.lower()

AttributeError: Can only use .str accessor with string values!

Removing Urls:

In [ ]:
url_pattern = re.compile(r'https?://\S+|www\.\S+')
data['text'] = data['text'].apply(lambda x: url_pattern.sub(r'', x))

Removing Mentions/ hashtags:

In [ ]:
mentions_pattern = re.compile(r'@\S+')
data['text'] = data['text'].apply(lambda x: mentions_pattern.sub(r'', x))

In [ ]:
hashs_pattern = re.compile(r'#\S+')
data['text'] = data['text'].apply(lambda x: hashs_pattern.sub(r'', x))

Removing Extra White Spaces:

In [ ]:
data['text'] = data['text'].apply(lambda x: re.sub(r'\s+', ' ', x))
data['text'] = data['text'].str.strip()

Removing Puntuation:

In [ ]:
pun = pun.replace("'", '')
pun = pun.replace(".", '')
pun

'!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~'

In [ ]:
translator = str.maketrans('', '', pun)                                     # Probably better to drop this step
data['text'] = data['text'].apply(lambda x: x.translate(translator))

Converting Abbreviations:

In [ ]:
pattern = r'\b(' + '|'.join(re.escape(key) for key in abbreviations.keys()) + r')\b'
data['text'] = data['text'].apply(lambda x: re.sub(pattern, lambda match: abbreviations[match.group(0).lower()], x))

Speeling Correction:

In [ ]:
#data['text'] = data['text'].apply(lambda x: str(TextBlob(x).correct()))

Removing Stopwords:

In [ ]:
data['text'] = data['text'].apply(lambda x: ' '.join(word for word in x.split() if word.lower() not in STOPWORDS))

Removing emojies:

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

data['text'] = data['text'].apply(lambda x: remove_emoji(x))


In [ ]:
data.to_csv('processed_data.csv', index=False)